# 11장 자연어처리 1부

## 11.1 자연어처리 소개

## 11.2 텍스트 벡터화

## 11.3 단어 모음 표현법: 집합과 시퀀스

여기서는 IMDB 영화 후기 데이터를 이용하여 두 모델 방식의
활용법과 차이점을 소개한다.

### 11.3.1 IMDB 영화 후기 데이터 준비

이전과는 달리 여기서는 IMDB 데이터셋을 직접 다운로드하여 전처리하는
과정을 살펴본다.

준비 과정 1: 데이터셋 다운로드 압축 풀기

압축을 풀면 아래 구조의 디렉토리가 생성된다.

```
aclImdb/
...train/
......pos/
......neg/
...test/
......pos/
......neg/
```

`train`의 `pos`와 `neg` 서브디렉토리에 각각 12,500개의 긍정과 부정 후기가
포함되어 있다.

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  49.8M      0  0:00:01  0:00:01 --:--:-- 49.8M


`aclImdb/train/unsup` 서브디렉토리는 필요 없기에 삭제한다.

In [ ]:
if 'google.colab' in str(get_ipython()):
    !rm -r aclImdb/train/unsup
else:
    import shutil
    unsup_path = './aclImdb/train/unsup'
    shutil.rmtree(unsup_path)

긍정 후기 하나의 내용을 살펴보자.
모델 구성 이전에 훈련 데이터셋을 살펴 보고
모델에 대한 직관을 갖는 과정이 항상 필요하다.

In [ ]:
if 'google.colab' in str(get_ipython()):
    !cat aclImdb/train/pos/4077_10.txt
else:
    with open('aclImdb/train/pos/4077_10.txt', 'r') as f:
        text = f.read()
        print(text)

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

준비 과정 2: 검증셋 준비

훈련셋의 20%를 검증셋으로 떼어낸다.
이를 위해 `aclImdb/val` 디렉토리를 생성한 후에
긍정과 부정 훈련셋 모두 무작위로 섞은 후 그중 20%를 검증셋 디렉토리로 옮긴다.

In [ ]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
    os.makedirs(val_dir / category)            # val 디렉토리 생성
    files = os.listdir(train_dir / category)

    random.Random(1337).shuffle(files)         # 훈련셋 무작위 섞기

    num_val_samples = int(0.2 * len(files))    # 20% 지정 후 검증셋으로 옮기기
    val_files = files[-num_val_samples:]

    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

준비 과정 3: 텐서 데이터셋 준비

`text_dataset_from_directory()` 함수를 이용하여
훈련셋, 검증셋, 테스트셋을 준비한다.
자료형은 모두 `Dataset`이며, 배치 크기는 32를 사용한다.

In [ ]:
from tensorflow import keras

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
    )

val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
    )

test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
    )

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


각 데이터셋은 배치로 구분되며
입력은 `tf.string` 텐서이고, 타깃은 `int32` 텐서이다.
크기는 모두 32이며 지정된 배치 크기이다.
예를 들어, 첫째 배치의 입력과 타깃 데이터의 정보는 다음과 같다.

In [ ]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)

    # 예제: 첫째 배치의 첫째 후기
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])

    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"Obnoxious Eva Longoria dies on her wedding day when an ice sculpture of an angel (without wings) falls on her off the back of a truck and kills her. She is then tries to ruin the relationship of her ex-boyfriend with his new girlfriend, a psychic who can see her.<br /><br />Obvious unoriginal movie wouldn't be bad in a clich\xc3\xa9d sort of way, except that Longoria's character is hateful and obnoxious that she drains all of the fun out of the film. Its like having your ears cleaned with sandpaper. To be fair Longoria, nor anyone else in the cast or crew, isn't the problem, its the god awful script that sinks the proceedings. Its just really really stupid.", shape=(), dtype=string)
targets[0]: tf.Tensor(0, shape=(), dtype=int32)


### 11.3.3 시퀀스 활용법

**정수 벡터 데이터셋 준비**

훈련셋의 모든 후기 문장을 정수들의 벡터로 변환한다.
단, 후기 문장이 최대 600개의 단어만 포함하도록 한다.
또한 사용되는 어휘는 빈도 기준 최대 2만 개로 제한한다.

- `max_length = 600`
- `max_tokens = 20000`
- `output_sequence_length=max_length`

In [ ]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000

text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)

text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y))
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y))

변환된 첫째 배치의 입력과 타깃 데이터의 정보는 다음과 같다.
`output_sequence_length=600`으로 지정하였기에 모든 문장은 단어를 최대 600개에서
잘린다. 따라서 생성되는 정수들의 벡터는 길이가 모두 600으로 지정된다.
물론 문장이 600개보다 적은 수의 단어를 사용한다면 나머지는 0으로 채워진다.
또한 벡터에 사용된 정수는 2만보다 작은 값이며,
이는 빈도가 가장 높은 2만개의 단어만을 대상(`max_tokens=20000`)으로 했기 때문이다.

In [ ]:
for inputs, targets in int_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 600)
inputs.dtype: <dtype: 'int64'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(
[ 1425   178   386   533     2   726     1  1778     1  5294     8     1
     3 10277    57    16  7197   173  2372     1    17    11  4355 12411
 13153  1625  8969  1778   286     4  1017  1590  4401  2449  9189   124
    40 10813    21    34  1074   115    54 14792    52  9942    40  9629
     6    34   931  3120  3696    51     1  3813   265    57  5569     6
    28     4  9125     3   160   457  1778  2533    40    62     7    57
     1     8     2  1570    30   867     6  8538    44  1778     3  3813
    24   114   911    42    44     9    14  1625  3572     6    94    65
   102    38  7724    44   172     2   404    92     2   114     7   522
  3813 17937    25   408   568     2   693  4963   430   211   131  1778
     1  5372   422    63   372    38     7 11498  8779    15 10721     1
     1     2  1593     5     1    13    13  3300  4963     7    43 

**트랜스포머 구현**

위 그림에서 설명된 트랜스포머 인코더를 층(layer)으로 구현하면 다음과 같다.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

**트랜스포머 인코더 활용 모델**

훈련 데이터셋이 입력되면 먼저 단어 임베딩을 이용하여
단어들 사이의 연관성을 찾는다.
이후 트랜스포머 인코더로 셀프 어텐션을 적용한다.

In [ ]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")

x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

# 길이가 600인 1차원 어레이로 변환
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 transformer_encoder (Trans  (None, None, 256)         543776    
 formerEncoder)                                                  
                                                                 
 global_max_pooling1d (Glob  (None, 256)               0         
 alMaxPooling1D)                                                 
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257   

훈련 과정은 특별한 게 없다.
테스트셋에 대한 정확도가 87.5% 정도로 바이그램 모델보다 좀 더 낮다.

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_encoder",
                                    save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=5, callbacks=callbacks)
# model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

model = keras.models.load_model(
    "transformer_encoder",
    custom_objects={"TransformerEncoder": TransformerEncoder})

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/5
625/625 [==============================] - 48s 77ms/step - loss: 0.3207 - accuracy: 0.8619 - val_loss: 0.3410 - val_accuracy: 0.8494
Epoch 2/5
625/625 [==============================] - 48s 77ms/step - loss: 0.2878 - accuracy: 0.8774 - val_loss: 0.3082 - val_accuracy: 0.8710
Epoch 3/5
625/625 [==============================] - 48s 78ms/step - loss: 0.2578 - accuracy: 0.8935 - val_loss: 0.3053 - val_accuracy: 0.8676
Epoch 4/5
625/625 [==============================] - 44s 70ms/step - loss: 0.2257 - accuracy: 0.9112 - val_loss: 0.3263 - val_accuracy: 0.8644
Epoch 5/5
782/782 [==============================] - 21s 26ms/step - loss: 0.3357 - accuracy: 0.8511
Test acc: 0.851


**단어 위치 인코딩**

다음 `PositionalEmbedding` 층 클래스는 두 개의 임베딩 클래스를 사용한다.
하나는 보통의 단어 임베딩이며,
다른 하나는 단어의 위치 정보를 임베딩한다.
각 임베딩의 출력값을 합친 값을 트랜스포머에게 전달하는 역할을 수행한다.

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**단어위치인식 트랜스포머 아키텍처**

아래 코드는 `PositionalEmbedding` 층을 활용하여 트랜스포머 인코더가
단어위치를 활용할 수 있도록 한다.
최종 모델의 테스트셋에 대한 정확도가 88.3%까지 향상됨을 확인할 수 있다.

In [ ]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder",
                                    save_best_only=True)
]
# model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)
model.fit(int_train_ds, validation_data=int_val_ds, epochs=5, callbacks=callbacks)
model = keras.models.load_model(
    "full_transformer_encoder",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding (Posi  (None, None, 256)         5273600   
 tionalEmbedding)                                                
                                                                 
 transformer_encoder_1 (Tra  (None, None, 256)         543776    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_1 (Gl  (None, 256)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                           

## 11.5. 시퀀스-투-시퀀스 학습

### A machine translation example

In [1]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

--2023-12-04 16:04:56--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.4.207, 142.251.10.207, 142.251.12.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.4.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  2.41MB/s    in 1.0s    

2023-12-04 16:04:57 (2.41 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]



In [2]:
text_file = "spa-eng/spa.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

In [3]:
import random
print(random.choice(text_pairs))

('I gave her just what she needed.', '[start] Le di justo lo que necesitaba. [end]')


In [4]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

**Vectorizing the English and Spanish text pairs**

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import string
import re

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

**Preparing training and validation datasets for the translation task**

In [7]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [8]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


### Sequence-to-sequence learning with Transformer

**The TransformerDecoder**

In [9]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

#### Putting it all together: a Transformer for machine translation

**PositionalEmbedding layer**

In [10]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**End-to-end Transformer**

In [13]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

**Training the sequence-to-sequence Transformer**

In [15]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

transformer.fit(train_ds, epochs=3, validation_data=val_ds)
# transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/3
1302/1302 [==============================] - 104s 76ms/step - loss: 3.7903 - accuracy: 0.4404 - val_loss: 2.8882 - val_accuracy: 0.5304
Epoch 2/3
1302/1302 [==============================] - 90s 69ms/step - loss: 2.8508 - accuracy: 0.5496 - val_loss: 2.4917 - val_accuracy: 0.5919
Epoch 3/3
1302/1302 [==============================] - 101s 77ms/step - loss: 2.5524 - accuracy: 0.5940 - val_loss: 2.3856 - val_accuracy: 0.6111


**Translating new sentences with our Transformer model**

In [16]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Tom isn't a good person.
[start] tom no es una persona [end]
-
Automobile sales suffered a setback at the end of the financial year.
[start] el examen de [UNK] a la luz al final de la año [end]
-
I don't need an explanation.
[start] no necesito una explicación [end]
-
Tom is going to college now.
[start] tom está a ir a la universidad ahora [end]
-
The dish is too sweet for Tom.
[start] la carta es demasiado bajo para tom [end]
-
I don't think you know anything.
[start] no sé que tú sabes nada [end]
-
How quickly does the bird fly?
[start] cómo se puede correr el pájaro [end]
-
Do you know what Tom's secret is?
[start] sabes lo que es el secreto de tom [end]
-
I'm going to cry.
[start] voy a ir a llorar [end]
-
The reason she killed herself is unknown.
[start] la razón se [UNK] a la razón [end]
-
If I were you, I'd go home right away.
[start] si yo fuera tú yo fuera a casa mismo [end]
-
Go to work.
[start] ve al trabajo [end]
-
I'll send you the link.
[start] te [UNK] la luz [end]
-
